In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import torch
import tensorflow as tf
import keras
import os, shutil 
# 原始目录所在的路径
original_dataset_dir = '/Users/a123/vscode/njupt-woc/catdogdata/train'


# 数据集分类后的目录    
base_dir = '/Users/a123/vscode/njupt-woc/catdogdata/cats_and_dogs_small' 
os.mkdir(base_dir)

# 训练、验证、测试数据集的目录
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 猫训练图片所在目录
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# 猫验证图片所在目录
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# 猫测试数据集所在目录
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# 狗训练图片所在目录
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# 狗验证数据集所在目录
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# 狗测试数据集所在目录
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [ ]:
label=[]
for img_digit in range(1,15626):
    if os.path.exists(f"/Users/a123/vscode/njupt-woc/catdogdata/train/cat.{img_digit}.jpg"):
        label.append(f"cat{img_digit}")
    elif os.path.exists(f"/Users/a123/vscode/njupt-woc/catdogdata/train/dog.{img_digit}.jpg"):
        label.append(f"dog{img_digit}")
label

In [ ]:
import re
catlist = []
doglist = []
for i in label:
    count = re.findall(r'\d+',i)
    count = int(count[0])
    if re.match(r'cat', i):
        catlist.append(count)
    else:
        doglist.append(count)
catlist


In [ ]:
doglist

In [ ]:
fnames = []
for i in label:
    for count in catlist:
        if re.match(r"cat", i):
            if count <= 10000:
                # 将前10000张猫图像复制到train_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(train_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []
                    

            if count > 10000 and count <= 12000:
                # 将下2000张猫图像复制到validation_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(validation_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 12000 and count <= 14000:
                # 将下2000张猫图像复制到test_cats_dir
                fnames.append('cat.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(test_cats_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

    for count in doglist:
        if re.match(r"dog",i):
            if count <= 10000:
                # 将前10000张狗图像复制到train_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(train_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 10000 and count <= 12000:
                # 将下2000张狗图像复制到validation_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(validation_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

            if count > 12000 and count <= 14000:
                # 将下2000张狗图像复制到test_dogs_dir
                fnames.append('dog.{}.jpg'.format(count))
            for fname in fnames:
                src = os.path.join(original_dataset_dir, fname)
                dst = os.path.join(test_dogs_dir, fname)
                shutil.copyfile(src, dst)
            fnames = []

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

In [3]:
# 此处改成自己的路径
train_dir='/Users/a123/vscode/njupt-woc/catdogdata/cats_and_dogs_small/train'
validation_dir='/Users/a123/vscode/njupt-woc/catdogdata/cats_and_dogs_small/validation'

In [7]:
# 网络架构
from keras import layers, models, regularizers
from keras.callbacks import EarlyStopping
from keras.applications import ResNet50
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers

# 图像预处理
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# 注意：验证集不应该进行数据增强
test_datagen = ImageDataGenerator(rescale=1./255)

# 加载预训练的 ResNet50 模型，不包括顶部的全连接层
conv_base = ResNet50(weights='/Users/a123/vscode/njupt-woc/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                     include_top=False,
                     input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)  # 将预训练的卷积基添加到新的模型中
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(Dropout(0.5))  # 使用 Dropout，dropout rate 为 0.5
model.add(layers.Dense(1, activation='sigmoid'))  # 添加新的全连接层来处理你的问题

# 解冻 conv_base，并冻结其中的部分层
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'res5c_branch2b':  # 从这一层开始解冻
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# 使用较低的学习率进行微调
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-5),
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=50,
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=50,
                                                        class_mode='binary')
history = model.fit(train_generator,
                    steps_per_epoch=200,
                    epochs=200,
                    validation_data=validation_generator,
                    validation_steps=40,
                    callbacks=[early_stopping])
model.save('cats_and_dogs_small_1.keras')

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/200
200/200 [==============================] - 528s 3s/step - loss: 0.7205 - accuracy: 0.5418 - val_loss: 0.6487 - val_accuracy: 0.6390
Epoch 2/200
200/200 [==============================] - 453s 2s/step - loss: 0.6723 - accuracy: 0.5864 - val_loss: 0.6349 - val_accuracy: 0.6565
Epoch 3/200
200/200 [==============================] - 448s 2s/step - loss: 0.6633 - accuracy: 0.6006 - val_loss: 0.6303 - val_accuracy: 0.6635
Epoch 4/200
200/200 [==============================] - 457s 2s/step - loss: 0.6595 - accuracy: 0.6080 - val_loss: 0.6223 - val_accuracy: 0.6725
Epoch 5/200
200/200 [==============================] - 455s 2s/step - loss: 0.6542 - accuracy: 0.6119 - val_loss: 0.6227 - val_accuracy: 0.6755
Epoch 6/200
200/200 [==============================] - 470s 2s/step - loss: 0.6492 - accuracy: 0.6196 - val_loss: 0.6086 - val_accuracy: 0.6760
Epoch 7/200
200/200 [==============================

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [25]:
from keras.preprocessing import image
import numpy as np
import os
from keras.models import load_model
import csv

with open('/Users/a123/vscode/njupt-woc/data/submission.csv','a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'TARGET'])
        csvfile.close()
model = load_model('cats_and_dogs_small_1.h5')
test_dir = '/Users/a123/vscode/njupt-woc/catdogdata/test'
fnames = []
list = []
m = 1
for i in os.listdir(test_dir):
    fnames.append(i)
for fname in fnames:
    img_path = os.path.join(test_dir, fname)
    img = image.load_img(img_path, target_size=(150, 150))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.  # 和训练时一样，需要进行归一化
    classes = model.predict(x)
    dogprobility = classes[0]
    with open('/Users/a123/vscode/njupt-woc/data/submission.csv', 'a') as csvfile:
        writer = csv.writer(csvfile, lineterminator='\n')
        writer.writerow([m, dogprobility[0]])
        csvfile.close()
    m += 1

1/1 [==============================] - 0s 48ms/step


KeyboardInterrupt: 